### Base de Datos : Citi Bike

+ Citi Bike es un sistema de intercambio de bicicletas públicas de gestión privada que sirve en parte de la ciudad de Nueva York.
+ Se dedidío trabajar cone sta base de datos para la optimización de ingresos dependiendo del tipo de pase que ofrece la compañía. 

### Diccionario de datos:

+ Trip Duration (seconds)
+ Start Time and Date
+ Stop Time and Date
+ Start Station Name
+ End Station Name
+ Station ID
+ Station Lat/Long
+ Bike ID
+ User Type (Customer = 24-hour pass or 3-day pass user; Subscriber = Annual Member)
+ Gender (Zero=unknown; 1=male; 2=female)
+ Year of Birth

Para tener la base de datos lista hicimos algunos supuestos para completarla de acuerdo a la información que necesitamos para modelarla, se crearan las siguientes variables:
+ Un id_trip - único por viaje
+ Una columna que desagrege *User Type* para tener los 3 tipos de pases que ofrece la compañia
+ Una columna que se llama *trip_category* para obtener si el viaje fue : *one way o round trip*

Estas columnas se crearon respetando el modelo del negocio y con la información de sus reportes mensuales, en este caso de Diciembre 2019.

In [246]:
cd notebooks/

/content/practica-final-opt-2021/notebooks


In [4]:
 #Descarga de datos
 #!curl -O https://s3.amazonaws.com/tripdata/201912-citibike-tripdata.csv.zip
 #!unzip 201912-citibike-tripdata.csv.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 39.1M  100 39.1M    0     0  21.6M      0  0:00:01  0:00:01 --:--:-- 21.6M
Archive:  201912-citibike-tripdata.csv.zip
  inflating: 201912-citibike-tripdata.csv  


In [248]:
import pandas as pd

data = pd.read_csv('../data/201912-citibike-tripdata.csv.zip')
data.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,602,2019-12-01 00:00:05.5640,2019-12-01 00:10:07.8180,3382,Carroll St & Smith St,40.680611,-73.994758,3304,6 Ave & 9 St,40.668127,-73.983776,41932,Subscriber,1970,1
1,1206,2019-12-01 00:00:10.9630,2019-12-01 00:20:17.8820,362,Broadway & W 37 St,40.751726,-73.987535,500,Broadway & W 51 St,40.762288,-73.983362,18869,Customer,1999,1
2,723,2019-12-01 00:00:11.8180,2019-12-01 00:12:14.8310,146,Hudson St & Reade St,40.716250,-74.009106,238,Bank St & Washington St,40.736197,-74.008592,15334,Subscriber,1997,1
3,404,2019-12-01 00:00:12.2200,2019-12-01 00:06:56.8860,3834,Irving Ave & Halsey St,40.694670,-73.906630,3827,Halsey St & Broadway,40.685650,-73.915640,41692,Customer,1995,1
4,1059,2019-12-01 00:00:14.7230,2019-12-01 00:17:54.1860,500,Broadway & W 51 St,40.762288,-73.983362,3323,W 106 St & Central Park West,40.798186,-73.960591,40156,Subscriber,1961,1


In [249]:
#Creación de la columna trip_id

#generamos los ID's
n= len(data.index)
digits_range = range(1345,n + 1345)
digits_list = list(digits_range)

#Agreagamos la columna al df
data['trip_id'] = digits_list

In [250]:
#Se obtienen las filas que tienen como usertype "Customer"
subset_c = data.loc[data['usertype'] == 'Customer']

#Se obtienen las filas que tienen como usertype "Subscriber"
subset_s = data.loc[data['usertype'] == 'Subscriber']

In [251]:
data['usertype'].unique()

array(['Subscriber', 'Customer'], dtype=object)

Esta parte es la que se tiene que dividir en 2 tipo de costumer:
+ costumer a =  (24-hour pass)
+ costumer b = 3-day pass user

De acuerdo a lo obtenido en el reporte de ese mes de CitiBike : https://d21xlh2maitm24.cloudfront.net/nyc/December-2019-Citi-Bike-Monthly-Report.pdf?mtime=20200218125724
Se tiene que en el 2019(12) en promedio tuvieron estos pases en cada una de las modalidades que manejan:
+ 10,816 single-day passes (91.5%)
+ 993 three-day passes (8.4%)

In [252]:
subset_c.shape[0]

#Número de pases de 1 día
one_day = round(subset_c.shape[0]*.915)

#Número de pases de 3 días
tree_day = subset_c.shape[0] - one_day

print("Numero de pasese en diciembre de 1 día",one_day )
print("Numero de pasese en diciembre de 3 días",tree_day )
one_day + tree_day

Numero de pasese en diciembre de 1 día 71027
Numero de pasese en diciembre de 3 días 6598


77625

In [253]:
import numpy as np

#Creación de columna auxiliar para separar los pases de Costumer en 2
subset_c['aux']=np.random.binomial(1,.915,77625)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [254]:
#Creación de columna con el nombre de los pases nuevos
subset_c['type_pass'] = np.where(subset_c['aux'] == 1, "one-day", "tree-days")
#Borramos la columa auxiliar
del subset_c['aux']

subset_s['type_pass']='annual'
#Juntamos los dos dataframes
data = subset_s.append(subset_c, ignore_index=True)
data.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


(955210, 17)

In [255]:
data.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,trip_id,type_pass
0,602,2019-12-01 00:00:05.5640,2019-12-01 00:10:07.8180,3382,Carroll St & Smith St,40.680611,-73.994758,3304,6 Ave & 9 St,40.668127,-73.983776,41932,Subscriber,1970,1,1345,annual
1,723,2019-12-01 00:00:11.8180,2019-12-01 00:12:14.8310,146,Hudson St & Reade St,40.716250,-74.009106,238,Bank St & Washington St,40.736197,-74.008592,15334,Subscriber,1997,1,1347,annual
2,1059,2019-12-01 00:00:14.7230,2019-12-01 00:17:54.1860,500,Broadway & W 51 St,40.762288,-73.983362,3323,W 106 St & Central Park West,40.798186,-73.960591,40156,Subscriber,1961,1,1349,annual
3,1257,2019-12-01 00:00:20.3490,2019-12-01 00:21:18.2990,340,Madison St & Clinton St,40.712690,-73.987763,340,Madison St & Clinton St,40.712690,-73.987763,40829,Subscriber,1994,1,1350,annual
4,624,2019-12-01 00:00:22.0410,2019-12-01 00:10:46.4270,469,Broadway & W 53 St,40.763441,-73.982681,526,E 33 St & 5 Ave,40.747659,-73.984907,24934,Subscriber,1983,1,1351,annual


In [256]:
data['type_pass'].unique()

array(['annual', 'one-day', 'tree-days'], dtype=object)

De acuerdo a lo obtenido en el reporte de CitiBike : https://d21xlh2maitm24.cloudfront.net/nyc/December-2019-Citi-Bike-Monthly-Report.pdf?mtime=20200218125724

Menciona que hay  43,395 single-trip pases (4.54%)



In [257]:
#Creación de una columna nueva : trip_category
data['aux']=np.random.binomial(1,0.0454,955210)

data['trip_category'] = np.where(data['aux'] == 1, "one-way", "round-trip")
#Borramos la columa auxiliar
del data['aux']
data.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,trip_id,type_pass,trip_category
0,602,2019-12-01 00:00:05.5640,2019-12-01 00:10:07.8180,3382,Carroll St & Smith St,40.680611,-73.994758,3304,6 Ave & 9 St,40.668127,-73.983776,41932,Subscriber,1970,1,1345,annual,round-trip
1,723,2019-12-01 00:00:11.8180,2019-12-01 00:12:14.8310,146,Hudson St & Reade St,40.716250,-74.009106,238,Bank St & Washington St,40.736197,-74.008592,15334,Subscriber,1997,1,1347,annual,round-trip
2,1059,2019-12-01 00:00:14.7230,2019-12-01 00:17:54.1860,500,Broadway & W 51 St,40.762288,-73.983362,3323,W 106 St & Central Park West,40.798186,-73.960591,40156,Subscriber,1961,1,1349,annual,round-trip
3,1257,2019-12-01 00:00:20.3490,2019-12-01 00:21:18.2990,340,Madison St & Clinton St,40.712690,-73.987763,340,Madison St & Clinton St,40.712690,-73.987763,40829,Subscriber,1994,1,1350,annual,round-trip
4,624,2019-12-01 00:00:22.0410,2019-12-01 00:10:46.4270,469,Broadway & W 53 St,40.763441,-73.982681,526,E 33 St & 5 Ave,40.747659,-73.984907,24934,Subscriber,1983,1,1351,annual,round-trip


In [258]:
#Convertimos la duración a minutos
data['duration'] = round(data['tripduration']/60)

In [260]:
data['starttime'] = pd.to_datetime(data['starttime']).dt.strftime('%Y-%m-%d %H:%M')
data['stoptime'] = pd.to_datetime(data['stoptime']).dt.strftime('%Y-%m-%d %H:%M')

In [261]:
data.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,trip_id,type_pass,trip_category,duration
0,602,2019-12-01 00:00,2019-12-01 00:10,3382,Carroll St & Smith St,40.680611,-73.994758,3304,6 Ave & 9 St,40.668127,-73.983776,41932,Subscriber,1970,1,1345,annual,round-trip,10.0
1,723,2019-12-01 00:00,2019-12-01 00:12,146,Hudson St & Reade St,40.716250,-74.009106,238,Bank St & Washington St,40.736197,-74.008592,15334,Subscriber,1997,1,1347,annual,round-trip,12.0
2,1059,2019-12-01 00:00,2019-12-01 00:17,500,Broadway & W 51 St,40.762288,-73.983362,3323,W 106 St & Central Park West,40.798186,-73.960591,40156,Subscriber,1961,1,1349,annual,round-trip,18.0
3,1257,2019-12-01 00:00,2019-12-01 00:21,340,Madison St & Clinton St,40.712690,-73.987763,340,Madison St & Clinton St,40.712690,-73.987763,40829,Subscriber,1994,1,1350,annual,round-trip,21.0
4,624,2019-12-01 00:00,2019-12-01 00:10,469,Broadway & W 53 St,40.763441,-73.982681,526,E 33 St & 5 Ave,40.747659,-73.984907,24934,Subscriber,1983,1,1351,annual,round-trip,10.0


### Referencias:
https://www.citibikenyc.com/system-data